In [1]:
datasets = sorted(["arguana", "dbpedia-entity", "germanquad", "msmarco", "quora", 
            "trec-covid", "climate-fever", "fever", "hotpotqa", "nfcorpus", "scidocs",  
            #"cqadupstack", 
            "fiqa", "nq", "scifact", "webis-touche2020"])

for i in datasets:
    print(i)

arguana
climate-fever
dbpedia-entity
fever
fiqa
germanquad
hotpotqa
msmarco
nfcorpus
nq
quora
scidocs
scifact
trec-covid
webis-touche2020


In [2]:
root = "/gallery_louvre/dayoon.ko/research/sds/src/datasets/"
import pandas as pd
for dataset in datasets:
    df = pd.read_csv(root + dataset + "/qrels/test.tsv", sep="\t")
    if "score" in df.columns:
        score_key = "score"
        corpus_key = "corpus-id"
    elif "rel" in df.columns:
        score_key = "rel"
        corpus_key = "c_id"
    else:
        print(df.columns)
    df = df[df[score_key] >= 1]
    counts = df[corpus_key].value_counts().values.mean()
    print(f"{dataset}: {round(counts, 2)} Q/D")

arguana: 1.0 Q/D
climate-fever: 3.48 Q/D
dbpedia-entity: 1.03 Q/D
fever: 5.29 Q/D
fiqa: 1.0 Q/D
germanquad: 4.42 Q/D
hotpotqa: 1.07 Q/D
msmarco: 1.0 Q/D
nfcorpus: 3.94 Q/D
nq: 1.0 Q/D
quora: 1.0 Q/D
scidocs: 1.23 Q/D
scifact: 1.2 Q/D
trec-covid: 1.41 Q/D
webis-touche2020: 1.01 Q/D


In [9]:
root = "/gallery_louvre/dayoon.ko/research/sds/src/datasets/"
import pandas as pd
for dataset in datasets:
    df = pd.read_csv(root + dataset + "/qrels/test.tsv", sep="\t")
    if "score" in df.columns:
        score_key = "score"
        corpus_key = "corpus-id"
    elif "rel" in df.columns:
        score_key = "rel"
        corpus_key = "c_id"
    df = df[df[score_key] >= 1]
    counts = [i for i in df[corpus_key].value_counts().values if i >=3]
    if len(counts) > 0:
        print(f"{dataset}: {len(counts)} documents with more than 2 related queries")

climate-fever: 275 documents with more than 2 related queries
dbpedia-entity: 60 documents with more than 2 related queries
fever: 715 documents with more than 2 related queries
germanquad: 314 documents with more than 2 related queries
hotpotqa: 84 documents with more than 2 related queries
nfcorpus: 1938 documents with more than 2 related queries
scidocs: 159 documents with more than 2 related queries
scifact: 10 documents with more than 2 related queries
trec-covid: 1422 documents with more than 2 related queries


In [7]:
from datasets import load_dataset

docs = load_dataset('irds/trec-robust04', 'docs')
#for record in docs:
#    record # {'doc_id': ..., 'text': ..., 'marked_up_doc': ...}

#queries = load_dataset('irds/trec-robust04', 'queries')
#for record in queries:
#    record # {'query_id': ..., 'title': ..., 'description': ..., 'narrative': ...}

#qrels = load_dataset('irds/trec-robust04', 'qrels')
#for record in qrels:
#    record # {'query_id': ..., 'doc_id': ..., 'relevance': ..., 'iteration': ...}

Generating docs split: 0 examples [00:00, ? examples/s]

[INFO] trec-robust04 is deprecated. Consider using disks45/nocr/trec-robust-2004 instead, which provides better parsing of the corpus.
[INFO] The TREC Robust document collection is from TREC disks 4 and 5. Due to the copyrighted nature of the  documents, this collection is for research use only, which requires agreements to be filed with NIST. See details here: <https://trec.nist.gov/data/cd45/index.html>.
More details about the procedure can be found here: <https://ir-datasets.com/trec-robust04.html#DataAccess>.
Once completed, place the uncompressed source here: /gallery_louvre/dayoon.ko/.ir_datasets/trec-robust04/trec45
This should contain directories like NEWS_data/FBIS, NEWS_data/FR94, etc.


DatasetGenerationError: An error occurred while generating the dataset